In [1]:
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

# Data

In [2]:
current_directory = os.getcwd()
current_directory

'/home/hermonpe/SIADS699/notebooks'

In [3]:
# Navigate to the parent directory
parent_directory = os.path.dirname(current_directory)
parent_directory

'/home/hermonpe/SIADS699'

### fpkm DATA

In [4]:
table = pq.read_table(parent_directory+'/data/processed/merged_df.parquet')

In [5]:
merged_df = table.to_pandas()

In [6]:
del table

In [7]:
merged_df.head()

,499304660,499304661,499304662,499304663,499304664,499304665,499304666,499304667,499304668,499304669,...,num_tbi_w_loc,dsm_iv_clinical_diagnosis,control_set,nincds_arda_diagnosis,ever_tbi_w_loc,race,hispanic,act_demented,braak,nia_reagan
rnaseq_profile_id,,,,,,,,,,,,,,,,,,,,,
488395315,0.655725,4.526404,0.0,0.0,0.039654,0.0,0.0,0.0,0.0,0.317608,...,0,No Dementia,3,No Dementia,N,White,Not Hispanic,No Dementia,3,2
496100277,0.095143,8.855850,0.0,0.0,0.016492,0.0,0.0,0.0,0.0,0.955393,...,0,No Dementia,14,No Dementia,N,White,Not Hispanic,No Dementia,3,1
496100278,0.000000,4.868456,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.361740,...,2,No Dementia,2,No Dementia,Y,White,Not Hispanic,No Dementia,6,2
496100279,0.000000,4.851842,0.0,0.0,0.170431,0.0,0.0,0.0,0.0,1.357990,...,2,No Dementia,2,No Dementia,Y,White,Not Hispanic,No Dementia,6,2
496100281,0.000000,3.600344,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.005507,...,0,Alzheimer's Disease Type,2,Probable Alzheimer'S Disease,N,White,Not Hispanic,Dementia,1,1


### Gene data

In [8]:
# gene information table
rows_genes = pd.read_csv(parent_directory+"/data/external/rows-genes.csv")
rows_genes = rows_genes.set_index('gene_id')
rows_genes

,chromosome,gene_entrez_id,gene_symbol,gene_name
gene_id,,,,
499304660,1,100287102,DDX11L1,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...
499304661,1,653635,WASH7P,WAS protein family homolog 7 pseudogene
499304662,1,102466751,MIR6859-1,microRNA 6859-1
499304663,1,100302278,MIR1302-2,microRNA 1302-2
499304664,1,645520,FAM138A,"family with sequence similarity 138, member A"
...,...,...,...,...
499355059,MT,4541,ND6,NADH dehydrogenase subunit 6
499355060,MT,4556,TRNE,tRNA-Glu
499355061,MT,4519,CYTB,cytochrome b


### Expression data subgroups by treatment

In [9]:
# separate the dataframe by treatment
dementia_df = merged_df[merged_df['act_demented']== 'Dementia']
nodementia_df = merged_df[merged_df['act_demented']!= 'Dementia']

df_dementia = dementia_df.iloc[:,:50281]
df_no_dementia = nodementia_df.iloc[:,:50281]

In [10]:
df_dementia.head()

,499304660,499304661,499304662,499304663,499304664,499304665,499304666,499304667,499304668,499304669,...,499355054,499355055,499355056,499355057,499355058,499355059,499355060,499355061,499355062,499355063
rnaseq_profile_id,,,,,,,,,,,,,,,,,,,,,
496100281,0.000000,3.600344,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.005507,...,367.205701,0.0,0.0,0.0,115.521316,43.398201,3.422902,259.190920,0.0,6.467808
496100283,0.000000,5.130615,0.0,0.0,0.007901,0.0,0.0,0.0,0.0,1.824261,...,580.226874,0.0,0.0,0.0,287.171532,60.780414,3.618383,476.317345,0.0,0.768925
496100284,0.000000,5.496470,0.0,0.0,0.137523,0.0,0.0,0.0,0.0,1.519029,...,353.021829,0.0,0.0,0.0,148.754839,47.585662,0.000000,284.561050,0.0,3.268784
496100285,0.151241,4.830538,0.0,0.0,0.087973,0.0,0.0,0.0,0.0,0.969436,...,376.928336,0.0,0.0,0.0,159.257390,39.506123,7.195433,300.710856,0.0,0.000000
496100294,0.459173,5.304398,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.070944,...,525.285614,0.0,0.0,0.0,183.870038,56.374182,2.077354,507.113073,0.0,0.000000


## Identifying Genes with Treatment-Induced Expression Changes

### T-test Statistical Analysis of Genes in No Dementia vs. Dementia Samples

In [11]:
# Perform t-test for each gene
t_statistic, p_value = ttest_ind(df_no_dementia, df_dementia, axis=0)

# Create a DataFrame with t-statistic and p-value for each gene
result_df = pd.DataFrame({
    't_statistic': t_statistic,
    'p_value': p_value
}, index=df_no_dementia.columns)

# merge gene information to t-statistic results
result_df.index = result_df.index.astype(int)
result_df = pd.merge(result_df, rows_genes, left_index=True, right_index=True)

In [12]:
result_df

,t_statistic,p_value,chromosome,gene_entrez_id,gene_symbol,gene_name
499304660,-0.776656,0.437852,1,100287102,DDX11L1,DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 l...
499304661,-1.692193,0.091440,1,653635,WASH7P,WAS protein family homolog 7 pseudogene
499304662,0.563897,0.573161,1,102466751,MIR6859-1,microRNA 6859-1
499304663,0.955770,0.339804,1,100302278,MIR1302-2,microRNA 1302-2
499304664,-1.506445,0.132795,1,645520,FAM138A,"family with sequence similarity 138, member A"
...,...,...,...,...,...,...
499355059,1.014898,0.310809,MT,4541,ND6,NADH dehydrogenase subunit 6
499355060,-0.253422,0.800081,MT,4556,TRNE,tRNA-Glu
499355061,1.907150,0.057265,MT,4519,CYTB,cytochrome b
499355062,-0.927079,0.354482,MT,4576,TRNT,tRNA-Thr


#### Calculate average expression change

In [13]:
# get average for each gene
dementia_mean = df_dementia.mean()
nondementia_mean = df_no_dementia.mean()

In [14]:
# Expression differences
expression_delta_values = nondementia_mean - dementia_mean
expression_delta_values = pd.Series(expression_delta_values, name='expression_delta')
expression_delta_values = expression_delta_values.abs()
expression_delta_values

499304660     0.015804
499304661     0.280732
499304662     0.002930
499304663     0.000590
499304664     0.009648
               ...    
499355059     1.898187
499355060     0.069123
499355061    32.857833
499355062     0.032852
499355063     0.376001
Name: expression_delta, Length: 50281, dtype: float64

In [15]:
expression_delta_values.index = expression_delta_values.index.astype(int)

In [16]:
# Export difference in expression by gene
expression_delta_values.to_csv(parent_directory+'/results/tables/expression_delta_values.csv')

In [17]:
# merge differences to results data frame
result_df = pd.merge(result_df, expression_delta_values, left_index=True, right_index=True)

In [18]:
#

In [19]:
result_df.sort_values(by='expression_delta', ascending=False)

,t_statistic,p_value,chromosome,gene_entrez_id,gene_symbol,gene_name,expression_delta
499339040,1.786002,0.074907,14,378706,RN7SL2,"RNA, 7SL, cytoplasmic 2",7979.260267
499339026,1.512811,0.131170,14,6029,RN7SL1,"RNA, 7SL, cytoplasmic 1",3714.845178
499338436,-1.947839,0.052180,14,85495,RPPH1,ribonuclease P RNA component H1,559.461635
499355049,2.498546,0.012898,MT,4514,COX3,cytochrome c oxidase subunit III,442.517488
499345143,-1.230173,0.219403,17,6066,RNU2-1,"RNA, U2 small nuclear 1",370.110317
...,...,...,...,...,...,...,...
499336369,NaN,NaN,12,100302217,MIR1827,microRNA 1827,0.000000
499351845,NaN,NaN,22,28808,IGLV3-2,immunoglobulin lambda variable 3-2 (pseudogene),0.000000
499351844,NaN,NaN,22,28786,IGLV4-3,immunoglobulin lambda variable 4-3,0.000000
499351843,NaN,NaN,22,28807,IGLV3-4,immunoglobulin lambda variable 3-4 (pseudogene),0.000000


In [20]:
# Set a significance threshold (e.g., 0.05)
significance_threshold = 0.05

# Filter genes with significant change
significant_genes_df = result_df[result_df['p_value'] < significance_threshold]

# Print the DataFrame with significant genes
print("DataFrame with significant genes:")
# print(significant_genes_df)
significant_genes_df.head(10)

DataFrame with significant genes:


,t_statistic,p_value,chromosome,gene_entrez_id,gene_symbol,gene_name,expression_delta
499304696,2.248627,0.025116,1,105378581,LOC105378581,uncharacterized LOC105378581,0.148896
499304697,1.996202,0.046635,1,100287934,LOC100287934,uncharacterized LOC100287934,0.121573
499304700,2.873385,0.004292,1,79854,LINC00115,long intergenic non-protein coding RNA 115,0.229860
499304703,-3.317771,0.000996,1,388579,TUBB8P11,"tubulin, beta 8 class VIII pseudogene 11",0.041285
499304708,-3.727154,0.000223,1,339451,KLHL17,kelch-like family member 17,0.191708
499304710,-2.116089,0.034996,1,84808,PERM1,"PPARGC1 and ESRR induced regulator, muscle 1",0.007201
499304714,-2.815131,0.005133,1,9636,ISG15,ISG15 ubiquitin-like modifier,1.057419
499304730,-2.316395,0.021075,1,51150,SDF4,stromal cell derived factor 4,0.474110
499304734,-2.253231,0.024822,1,101928895,LOC101928895,uncharacterized LOC101928895,0.170160
499304735,-2.949336,0.003384,1,6339,SCNN1D,"sodium channel, non voltage gated 1 delta subunit",0.203808


In this preliminary assessment, we adopted a notably stringent threshold of 0.001. This choice aims to yield a more restricted list of candidate genes, facilitating manual inspection and assessment. It is important to note that opting for such a low threshold may increade the risk of encountering Type II errors.

In [21]:
print('The toal number of gens with a significant difference at the selected threshold is:',len(significant_genes_df))

The toal number of gens with a significant difference at the selected threshold is: 3924


# Data Spot Checks

In [22]:
gene_to_check = 'APOE'

# Check if gene is in the gene_symbol column
is_apoe_present = gene_to_check in rows_genes['gene_symbol'].values

if is_apoe_present:
    print(f"{gene_to_check} is present in the DataFrame.")
else:
    print(f"{gene_to_check} is not present in the DataFrame.")
    


APOE is present in the DataFrame.


In [23]:
result_df[result_df['gene_symbol']==gene_to_check]

,t_statistic,p_value,chromosome,gene_entrez_id,gene_symbol,gene_name,expression_delta
499348657,0.794726,0.427276,19,348,APOE,apolipoprotein E,2.844767
